In [ ]:
from collections import defaultdict

In [ ]:
def parse_input(input_lines):
    robots = []
    for line in input_lines:
        # Split position and velocity parts
        p_part, v_part = line.split(' ')
        # Parse position coordinates
        px, py = map(int, p_part.split('=')[1].split(','))
        # Parse velocity components
        vx, vy = map(int, v_part.split('=')[1].split(','))
        robots.append(((px, py), (vx, vy)))
    return robots

In [ ]:
def wrap_position(pos, vel, width, height):
    x, y = pos
    # Apply movement
    x = (x + vel[0]) % width
    y = (y + vel[1]) % height
    return (x, y)

In [ ]:
def simulate(robots, seconds, width, height):
    final_positions = []
    for pos, vel in robots:
        current_pos = pos
        # Apply movement for given number of seconds
        for _ in range(seconds):
            current_pos = wrap_position(current_pos, vel, width, height)
        final_positions.append(current_pos)
    return final_positions

In [ ]:
def get_pos_in_quadrants(positions, width, height):
    max_x = width - 1
    max_y = height - 1
    quadrants = defaultdict(int)

    for pos in positions:
        x, y = pos
        if x < max_x // 2 and y < max_y // 2:
            quadrants[0] += 1
        if x > max_x // 2 and y < max_y // 2:
            quadrants[1] += 1
        if x < max_x // 2 and y > max_y // 2:
            quadrants[2] += 1
        if x > max_x // 2 and y > max_y // 2:
            quadrants[3] += 1

    result = 1
    for quadrant in quadrants:
        result *= quadrants[quadrant]

    return result

In [ ]:
file = "example"
height = 7
width = 11

In [ ]:
file = "input"
height = 103
width = 101

In [ ]:
with open(file) as f:
    lines = f.readlines()

In [ ]:
robots = parse_input(lines)
final_positions = simulate(robots, 100, width, height)

In [ ]:
get_pos_in_quadrants(final_positions, width, height)

Is the "christmas tree" symmetrical?

In [ ]:
def is_vertically_symmetric(positions, width):
    # Convert positions to set for O(1) lookups
    pos_set = set(positions)

    # Check each position has matching reflection
    for x, y in positions:
        reflected_x = width - 1 - x
        if (reflected_x, y) not in pos_set:
            return False

    return True

In [ ]:
def find_first_symmetry(robots, width, height):
    positions = [pos for pos, _ in robots]
    velocities = [vel for _, vel in robots]

    # Simulate until symmetry found
    for second in range(1000000000):  # Large upper bound
        # Move all robots one step
        positions = [wrap_position(pos, vel, width, height) for pos, vel in zip(positions, velocities)]

        # Check for symmetry
        if is_vertically_symmetric(positions, width):
            return second

    return None

Is the "christmas tree" a cluster of robots?

In [ ]:
def get_adjacent_positions(pos):
    x, y = pos
    return {
        (x+dx, y+dy)
        for dx in [-1,0,1]
        for dy in [-1,0,1]
        if not (dx == 0 and dy == 0)
    }

In [ ]:
def find_clusters(positions):
    clusters = []
    visited = set()

    def dfs(pos, cluster):
        cluster.add(pos)
        visited.add(pos)
        for adj in get_adjacent_positions(pos):
            if adj in positions and adj not in visited:
                dfs(adj, cluster)

    for pos in positions:
        if pos not in visited:
            cluster = set()
            dfs(pos, cluster)
            if len(cluster) > 1:  # Only track clusters of 2+ robots
                clusters.append(cluster)

    return clusters

In [ ]:
def find_first_cluster_threshold(robots, width, height, threshold=0.4):
    positions = [pos for pos, _ in robots]
    velocities = [vel for _, vel in robots]
    robot_count = len(robots)
    min_cluster_size = int(robot_count * threshold)

    for second in range(1000000):
        print(second)
        # Move all robots one step
        new_states = [(wrap_position(pos, vel, width, height), vel)
                     for pos, vel in zip(positions, velocities)]
        positions = [pos for pos, _ in new_states]
        velocities = [vel for _, vel in new_states]

        # Check for clusters
        clusters = find_clusters(positions)
        if any(len(cluster) >= min_cluster_size for cluster in clusters):
            return second + 1

    return None

In [ ]:
robots = parse_input(lines)
min_seconds = find_first_cluster_threshold(robots, width, height)
print(f"First large cluster occurs at {min_seconds + 1} seconds")